### EXTRAÇÃO DOS BALANCETES E BALANÇOS DAS IFS REGULADAS PELO BANCO CENTRAL DO BRASIL (BCB)

- **Nome oficial da base:** Balancetes e Balanços Patrimoniais
- **Órgão:** CMN, BCB
- **Periodicidade:** mensal
- **Período:** 1988 a 2022 
- **Forma de extração:** webscraping
- **Link:** hhttps://www.bcb.gov.br/estabilidadefinanceira/balancetesbalancospatrimoniais<td><td>

**Por Luis Adolfo Mazini Rodrigues¹ e Isabella de Moura Rocha²**

- ¹Graduado em Engenharia Mecânica, pela USP São Carlos e Doutorando em Bioenergia, pela USP Piracicaba, luis.mazinirodrigues@gmail.com 
- ²Graduada em Administração, pela UFSCar Sorocaba e Mestra em Administração, pela UFSCar Sorocaba, isabellamrocha0@gmail.com

*Os dados são base da dissertação: ROCHA, I.M. Rentabilidade, eficiência e risco em cooperativas de crédito e bancos comerciais. 2023. Dissertação (Mestrado em Administração) – Universidade Federal de São Carlos, Sorocaba, 2023. Disponível em: https://repositorio.ufscar.br/handle/ufscar/18956.*

### 1. Bibliotecas necessárias

São importadas as bibliotecas necessárias para o scraping da web e processamento dos dados das instituições financeiras.

In [1]:
import pandas as pd
import numpy as np
import wget
from zipfile import ZipFile
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import csv
from datetime import datetime
import time

### 2. Coleta de dados

Nesta seção, é realizado o scraping da web para baixar os balancetes das instituições.

In [2]:
class BCdata:
    def __init__(self):
        self.profile = webdriver.FirefoxProfile() # inicializa o perfil do navegador
        self.profile.set_preference("browser.download.folderList", 2) # define as preferências
        self.profile.set_preference("browser.download.manager.showWhenStarting", False)
        self.profile.set_preference("browser.download.dir", "/home/isabella/Documents/Masters/DATA/cooperativas_bancos/")
        self.profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/octet-stream")
        self.bot = webdriver.Firefox(firefox_profile = self.profile, executable_path = '/usr/local/bin/geckodriver')
    def downloadNow(self, list_value, list_id):
        bot = self.bot # obtém o navegador
        bot.get('https://www4.bcb.gov.br/fis/cosif/balancetes.asp?frame=1')
        i = 1
        for ids in list_id: # itera sobre os IDs 
            sel = Select(bot.find_element_by_xpath('//*[@id="' + ids + '"]'))
            i+=1
            for text in list_value:
                try:
                    str_search = '/fis/cosif/cont/balan/' + ids.lower() + '/' + text + ids.upper() # procura pelo arquivo zip no site
                    try:
                        sel.select_by_value(str_search + '.ZIP') # tenta selecionar o arquivo ZIP
                    except:
                        sel.select_by_value(str_search + '.zip') # se falhar, tenta com minúsculas
                    gotit = bot.find_element_by_xpath('/html/body/div/table[1]/tbody/tr[' + str(i) + ']/td[3]/input');  # clica em download
                    gotit.click()
                except:
                    pass

In [2]:
list_value = [] # inicializa lista vazia para armazenagem
list_id = ['Bancos', 'Cooperativas'] # define uma lista de IDs (bancos e cooperativas) para usar no método downloadNow
for ano in range(1988, 2022): # primeiro e último anos disponíveis
    for mes in range(1, 13): # meses de 1 a 12
        if mes < 10:  # se mês for menor do que 10 então adicione um zero, para manter o formato 'YYYYMM'
            list_value.append(str(ano) + '0' + str(mes))
        else: # se mês for maior do que 10, apenas concatena ano e mês
            list_value.append(str(ano) + str(mes))

In [4]:
bc = BCdata() # cria uma instância da classe BCdata
bc.downloadNow(list_value, list_id) # chama o método downloadNow da instância bc, e passa as listas list_value e list_id como argumentos

/tmp/ipykernel_9529/2935450669.py:3: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  self.profile = webdriver.FirefoxProfile()
/tmp/ipykernel_9529/2935450669.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.bot = webdriver.Firefox(firefox_profile = self.profile, executable_path = '/usr/local/bin/geckodriver')
/tmp/ipykernel_9529/2935450669.py:8: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  self.bot = webdriver.Firefox(firefox_profile = self.profile, executable_path = '/usr/local/bin/geckodriver')
/tmp/ipykernel_9529/2935450669.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  sel = Select(bot.find_element_by_xpath('//*[@id="' + ids + '"]'))
/tmp/ipykernel_9529/2935450669.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  gotit = bot.find_e

In [3]:
arquivos_zip = [] # inicializa lista vazia para armazenagem dos arquivos zip
str_file = '' # inicializa uma string vazia para armazenar o nome do arquivo temporariamente
for instituto in list_id: # loop pelos ids das instituições em list_id
    for ano in range(2001, 2022): # quero apenas esses anos agora
        for mes in range(1, 13): # mesmas condições da verificação anterior
            if mes < 10:
                str_file = str(ano) + '0' + str(mes) + instituto.upper()
            else:
                str_file = str(ano) + str(mes) + instituto.upper()
            arquivos_zip.append(str_file)    

### 3. Pré-processamento de Dados

Nesta seção, os arquivos são dezipados e tratados para limpeza e organização.

In [5]:
new_arq = list() # nova lista
header = ['#DATA_BASE','DOCUMENTO','CNPJ','AGENCIA','NOME_INSTITUICAO','COD_CONGL','NOME_CONGL','TAXONOMIA','CONTA','NOME_CONTA','SALDO'] # cabeçalho do arquivo csv
for arq in arquivos_zip: 

    if 'bancos' in arq.lower(): # loop pelos nomes de arquivo na lista arquivos_zip
        instituicao = 'BANCOS'  # se o nome do arquivo contiver 'bancos', então retorne 'BANCOS'
    else:
        instituicao = 'COOPERATIVAS' # se não, retorne 'COOPERATIVAS'
    try:
        try:  # extrai o arquivo ZIP para o diretório da instituição financeira e adiciona o nome do arquivo à lista
            ZipFile('/home/isabella/Documents/Masters/DATA/cooperativas_bancos/' + arq + '.ZIP').extractall('/home/isabella/Documents/Masters/DATA/cooperativas_bancos/CSV/' + instituicao)
            new_arq.append('/home/isabella/Documents/Masters/DATA/cooperativas_bancos/CSV/' + instituicao + '/' + arq + '.CSV')
        except:
            ZipFile('/home/isabella/Documents/Masters/DATA/cooperativas_bancos/' + arq + '.zip').extractall('/home/isabella/Documents/Masters/DATA/cooperativas_bancos/CSV/' + instituicao)
            new_arq.append('/home/isabella/Documents/Masters/DATA/cooperativas_bancos/CSV/' + instituicao + '/' + arq + '.CSV')
    except:
        pass

In [6]:
for arq in new_arq:
    with open(arq, 'r', encoding='ISO8859-1') as readFile:
        lines = list(csv.reader(readFile, delimiter=';'))
        while True: # procura a linha que contém '#DATA_BASE' ou 'CNPJ'
            if '#DATA_BASE' in lines[0] or 'CNPJ' in lines[0]:
                break
            else:
                del lines[0] # se não encontrar, dropa a primeira linha
        lines[0][0] = 'DATA'
    with open(arq, 'w', encoding='ISO8859-1') as writeFile:
        writer = csv.writer(writeFile, delimiter='|')
        writer.writerows(lines)

In [7]:
bancos_arq = list()
cooperativas_arq = list()

for arq in new_arq: # cada arquivo é adicionado a sua respectiva instituição financeira
    if 'csv/bancos' in arq.lower():
        bancos_arq.append(arq)
    else:
        cooperativas_arq.append(arq)

In [9]:
df_bancos = pd.concat(map(lambda x: pd.read_csv(x, sep='|', decimal=',', encoding='ISO8859-1'), bancos_arq)) # lê cada arquiv csv da lista e utiliza esses parâmetros para garantir a leitura correta, após isso concatena todos

In [10]:
df_cooperativas = pd.concat(map(lambda x: pd.read_csv(x, sep='|', decimal=',', encoding='ISO8859-1'), cooperativas_arq)) # lê cada arquiv csv da lista e utiliza esses parâmetros para garantir a leitura correta, após isso concatena todos

In [11]:
df_bancos.to_csv('allBancos.csv')

In [12]:
df_cooperativas.to_csv('allCoop.csv')

In [14]:
df_cooperativas["NOME_INSTITUICAO"] = np.where(df_cooperativas["NOME_INSTITUICAO"].isnull(), df_cooperativas["NOME INSTITUICAO"], df_cooperativas["NOME_INSTITUICAO"])

In [18]:
df_cooperativas = df_cooperativas.drop('NOME INSTITUICAO', axis=1, inplace=True)

In [ ]:
df_cooperativas['NOME INSTITUICAO'] = df_cooperativas.loc[df_cooperativas['NOME INSTITUICAO'].isnull(),'value_is_NaN'] = df_cooperativas['NOME_INSTITUICAO']

In [ ]:
BC_concat = df.drop('REALIZAVEL ATE 3M', axis=1, inplace=True)

In [ ]:
nomes = ['BPA_con', 'BPA_ind', 'BPP_con', 'BPP_ind', 'DFC_MD_con', 'DFC_MD_ind', 'DFC_MI_con', 'DFC_MI_ind', 'DMPL_con', 'DMPL_ind', 'DRE_ind', 'DRE_con', 'DRE_ind', 'DVA_con', 'DVA_ind']
for nome in nomes: 
    arquivo = pd.DataFrame()
    for ano in range(2011, 2022):
        arquivo = pd.concat([arquivo, pd.read_csv(f'Documents/Masters/DATA/CVM/itr_cia_aberta_{nome}_{ano}f.csv', sep=';', decimal=',', encoding='ISO8859-1')])
    arquivo.to_csv(f'Documents/Masters/DATA/itr_cia_aberta_{nome}_2011-2021.csv', index = False)